# Compiling TF1.15 (CV) Detection Models with SageMaker Neo

You need to run this notebook on a SageMaker Studio Instance for a complete experience!

**SageMaker Studio Kernel**: Data Science

TF1 model zoo: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf1_detection_zoo.md

We're going to use a **SSDMobilenetV1** in this example, but you can apply the same strategy used in this notebook to compile any models from the list. Depending on the model, you only need to customize the testing code in the last section.



In this exercise you'll:
   - Get a pre-trained model from the Model Zoo
   - Prepare the model to compile it with Neo
   - Compile the model for the target: **X86_64**
   - Get the optimized model and run a simple local test

In [ ]:
# required for local tests
!apt -y update && apt-get -y install libgl1
!pip install dlr opencv-python

In [ ]:
from sagemaker import get_execution_role

sagemaker_role = get_execution_role()

## 1) Get the pre-trainded model and upload it to S3

In [ ]:
import io
import os
import shutil
import tarfile
import sagemaker
import urllib.request

url='http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz'
model_name='ssd-mobilenetv1-coco'
img_size=300

print(f"Downloading the model: {url}")

if os.path.isdir('export'): shutil.rmtree('export')
with urllib.request.urlopen(url) as f:
    with tarfile.open(fileobj=io.BytesIO(f.read()), mode="r:gz") as tar:        
        tar.extractall(path='export')
        
print("Create a model package and upload it to S3")
sagemaker_session = sagemaker.Session()
with tarfile.open("model.tar.gz", "w:gz") as f:
    f.add(f"export/ssd_mobilenet_v1_coco_2018_01_28/saved_model/", "export/1")
    f.list()

s3_uri = sagemaker_session.upload_data('model.tar.gz', key_prefix=f'{model_name}/model')

print(f"Done\nS3 uri: {s3_uri}")

## 2) Compile the model with SageMaker Neo (X86_64)

**ATTENTION:** It takes around 30mins to compile an EfficientDet

In [ ]:
import time
import boto3
import sagemaker

arch='X86_64' # Jetson = ARM64

role = sagemaker.get_execution_role()
sm_client = boto3.client('sagemaker')
compilation_job_name = f'{model_name}-tf1-{int(time.time()*1000)}'
sm_client.create_compilation_job(
    CompilationJobName=compilation_job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': s3_uri,
        'DataInputConfig': f'{{"image_tensor": [1,{img_size},{img_size},3]}}',
        'Framework': 'TENSORFLOW',
        'FrameworkVersion': '1.15'
    },
    OutputConfig={
        'S3OutputLocation': f's3://{sagemaker_session.default_bucket()}/{model_name}-tf1/optimized/',
        'TargetPlatform': { 
            'Os': 'LINUX', 
            'Arch': arch,
            #'Accelerator': 'NVIDIA'  # comment this if you don't have an Nvidia GPU
        },
        # Comment or change the following line depending on your edge device
        # Jetson Xavier: sm_72; Jetson Nano: sm_53
        #'CompilerOptions': '{"trt-ver": "7.1.3", "cuda-ver": "10.2", "gpu-code": "sm_72"}' # Jetpack 4.4.1
    },
    StoppingCondition={ 'MaxRuntimeInSeconds': 18000 }
)
while True:
    resp = sm_client.describe_compilation_job(CompilationJobName=compilation_job_name)    
    if resp['CompilationJobStatus'] in ['STARTING', 'INPROGRESS']:
        print('Running...')
    else:
        print(resp['CompilationJobStatus'], compilation_job_name)
        break
    time.sleep(5)
    

## 3) Download the compiled model
**ATTENTION:** Only for X86_64 with no GPU

In [ ]:
output_model_path = f's3://{sagemaker_session.default_bucket()}/{model_name}-tf1/optimized/model-LINUX_{arch}.tar.gz'
!aws s3 cp $output_model_path /tmp/model.tar.gz
!rm -rf compiled_model && mkdir compiled_model
!tar -xzvf /tmp/model.tar.gz -C compiled_model

## 4) Run the model locally

### download the labels and a sample image

In [ ]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import urllib.request

labels_url='https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/object_detection/ios/ObjectDetection/Model/labelmap.txt'
image_url='https://sagemaker-examples.readthedocs.io/en/latest/_images/cat2.jpg'
if not os.path.exists('coco_labels.txt'):
    urllib.request.urlretrieve(labels_url, 'coco_labels.txt')
if not os.path.exists('cat.jpg'):
    urllib.request.urlretrieve(image_url, 'cat.jpg')
labels = [i.strip() for i in open('coco_labels.txt', 'r').readlines()]

### load the model using the runtime DLR

In [ ]:
import dlr
# load the model (CPU x86_64)
model = dlr.DLRModel('compiled_model', 'cpu')

In [ ]:
# load the image and make it squared if needed
img = cv2.cvtColor(cv2.imread('cat.jpg'), cv2.COLOR_BGR2RGB)
x = cv2.resize(img, (img_size,img_size))
h,w,c = x.shape
x = x.reshape(1,h,w,c) # HWC  --> NHWC
x.shape

In [ ]:
## Compute times
y = model.run(x) # warm-up
%timeit model.run(x)

In [ ]:
y = model.run(x)
img_ = img.copy()
img_id=0
h,w,c = img_.shape
for bboxes, classes, scores in [(y[2][img_id], y[0][img_id], y[3][img_id])]:
    for bbox,class_id, score in zip(bboxes, classes,scores):
        if score > 0.6:            
            print(f"Class id: {class_id}, Score: {score}, Label: {labels[int(class_id)]}")
            x1,y1,x2,y2 = (bbox * (w,h,w,h)).astype(np.int32)            
            cv2.rectangle(img_, (x1,y1), (x2,y2), (255,0,0), 5)
plt.imshow(img_)

# Done! :)